In [ ]:
!pip install ortools

In [ ]:
from ortools.sat.python import cp_model
import pandas as pd
from typing import Dict, List
from collections import defaultdict
import re
from itertools import combinations

In [ ]:
def merge_data(df_registration, df_courses):
    df_merged = pd.merge(df_registration, df_courses, left_on='G CODE', right_on='Course code', how='left')
    df_merged['Professor'] = df_merged['Faculty Name']
    df_merged = df_merged[['Roll No.','G CODE', 'Professor']]
    return df_merged.dropna()

def prepare_student_course_map(df):
    return {roll: list(group['G CODE']) for roll, group in df.groupby('Roll No.')}

def create_course_professor_map(df):
    return pd.Series(df['Professor'].values, index=df['G CODE']).to_dict()


In [ ]:
def faculty_busy_slots(df_faculty_pref):
    return df_faculty_pref.groupby("Name")["Busy Slot"].agg(list).to_dict()

def create_course_dictionary(student_course_map, course_professor_map, professor_busy_slots):
    unique_courses = set().union(*student_course_map.values())
    course_availability = {}
    for course in unique_courses:
        professor = course_professor_map.get(course, None)
        busy_slots = professor_busy_slots.get(professor, [])
        all_slots = [f'{day} {time}' for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
                     for time in ['08:30', '10:30', '12:30', '14:30', '16:30', '18:30']]

        # Add specific exclusions for Wednesday at 2:30 PM and Tuesday at 2:30 PM
        excluded_slots = {'Wednesday 14:30', 'Tuesday 14:30'}

        # Filter slots by checking both professor busy slots and the excluded slots
        course_availability[course] = {
            'time_slots': [slot for slot in all_slots if slot not in busy_slots and slot not in excluded_slots]
        }
    return course_availability

def create_course_credit_map(df_courses):
    return pd.Series(df_courses['Credits'].apply(lambda x: 1 if x in [1, 2] else 2).values, index=df_courses['Course code']).to_dict()



In [ ]:
from ortools.sat.python import cp_model
import pandas as pd
from typing import Dict, List
from collections import defaultdict
import re
from itertools import combinations

def get_day_from_time_slot(time_slot):
    """
    Extracts the day from the time slot string.
    Assumes that the day is the first word in the time slot string.
    Modify this function based on the actual format of your time slots.
    """
    # Example time slot format: "Monday 9am-10am"
    day = time_slot.split()[0]
    return day

def schedule_courses(courses: Dict[str, Dict[str, List[str]]],
                     student_course_map: Dict[str, List[str]],
                     course_professor_map: Dict[str, str],
                     course_credits: Dict[str, int]) -> pd.DataFrame:
    """
     Schedules courses for a given set of courses, students, and professors,
    ensuring each course is scheduled exactly twice without student conflicts,
    while considering time slot preferences and limiting classes per time slot.
    
    Parameters:
    - courses (dict): A dictionary where each key is a course ID and its value
                      is a dictionary containing course details, including
                      available time slots.
    - student_course_map (dict): A mapping of student roll numbers to their
                                  enrolled courses.
    - course_professor_map (dict): A mapping of course IDs to their respective
                                    professors.
    Returns:
    - pd.DataFrame: A DataFrame containing scheduled courses with their
                    corresponding time slots, or an empty DataFrame if no
                    feasible solution is found.
    """

    # Initialize the constraint programming model
    model = cp_model.CpModel()

    # List of all time slots
    all_time_slots = set()
    for course_info in courses.values():
        all_time_slots.update(course_info['time_slots'])
    all_time_slots = sorted(all_time_slots)

    # Create variables for course schedules
    # course_time_vars[course_id][time_slot] = BoolVar
    course_time_vars = {}
    # Initialize time slot count vars
    time_slot_count_vars = defaultdict(list)

    for course_id, course_info in courses.items():
        time_vars = {}
        for time_slot in course_info['time_slots']:
            var = model.NewBoolVar(f'{course_id}_{time_slot}')
            time_vars[time_slot] = var
            # Collect variables per time slot for counting
            time_slot_count_vars[time_slot].append(var)
        course_time_vars[course_id] = time_vars

    # Constraint: Schedule each course as per its credits
    for course_id, time_vars in course_time_vars.items():
        if course_id in course_credits:
            model.Add(sum(time_vars.values()) == course_credits[course_id])
        else:
            print(f"Warning: No credit information for course {course_id}. Defaulting to 2 sessions.")
            model.Add(sum(time_vars.values()) == 2)  # defaulting to 2 if no credit info is provided

    # Professor conflict constraints (if a professor teaches multiple courses)
    professor_courses = defaultdict(list)
    for course_id, professor in course_professor_map.items():
        professor_courses[professor].append(course_id)
    for professor, courses_taught in professor_courses.items():
        # For each time slot, ensure the professor is not scheduled for more than one course
        time_slot_vars = defaultdict(list)
        for course_id in courses_taught:
            if course_id in course_time_vars:
                for time_slot, var in course_time_vars[course_id].items():
                    time_slot_vars[time_slot].append(var)
        for time_slot, vars_in_slot in time_slot_vars.items():
            if len(vars_in_slot) > 1:
                model.AddAtMostOne(vars_in_slot)

    # Student conflict constraints
    # For each student, introduce penalty variables for potential conflicts
    conflict_penalty_vars = []

    for student_id, enrolled_courses in student_course_map.items():
        # For each time slot, collect variables for the courses the student is enrolled in
        time_slot_vars = defaultdict(list)
        for course_id in enrolled_courses:
            if course_id in course_time_vars:
                for time_slot, var in course_time_vars[course_id].items():
                    time_slot_vars[time_slot].append(var)
        # For each time slot, create a conflict penalty variable
        for time_slot, vars_in_slot in time_slot_vars.items():
            if len(vars_in_slot) > 1:
                # Create a conflict variable that is 1 if there's a conflict at this time slot
                conflict_var = model.NewBoolVar(f'conflict_{student_id}_{time_slot}')
                # The conflict variable is true if the sum of scheduled courses at this time is >= 2
                # Add implication constraints
                model.Add(sum(vars_in_slot) >= 2).OnlyEnforceIf(conflict_var)
                model.Add(sum(vars_in_slot) <= 1).OnlyEnforceIf(conflict_var.Not())
                # Collect conflict variables to minimize in the objective function
                conflict_penalty_vars.append(conflict_var)

    # Add constraint to limit classes per time slot to a maximum of 15
    for time_slot, vars_in_slot in time_slot_count_vars.items():
        model.Add(sum(vars_in_slot) <= 15)

    # Soft Constraint: Avoid scheduling the same course more than once on the same day
    penalty_vars = []

    for course_id, time_vars in course_time_vars.items():
        # Mapping from day to time slot variables
        day_time_vars = defaultdict(list)
        for time_slot, var in time_vars.items():
            day = get_day_from_time_slot(time_slot)
            day_time_vars[day].append(var)
        for day, vars_on_day in day_time_vars.items():
            if len(vars_on_day) > 1:
                num_classes_on_day = model.NewIntVar(0, len(vars_on_day), f'num_classes_{course_id}_{day}')
                model.Add(num_classes_on_day == sum(vars_on_day))
                penalty_var = model.NewBoolVar(f'penalty_{course_id}_{day}')
                # penalty_var = 1 if num_classes_on_day >= 2
                model.Add(num_classes_on_day >= 2).OnlyEnforceIf(penalty_var)
                model.Add(num_classes_on_day <= 1).OnlyEnforceIf(penalty_var.Not())
                # Collect penalty_var to be added to the objective function
                penalty_vars.append(penalty_var)

    # Add the Objective Function to Minimize Conflicts and Penalties
    # You can adjust the weights to prioritize one over the other
    model.Minimize(1000 * sum(conflict_penalty_vars) + 750 * sum(penalty_vars))

    # Solve the model
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 15 * 60  
    solver.parameters.num_search_workers = 10
    status = solver.Solve(model)

    # Creating a DataFrame to hold the schedule
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        data = []
        for course_id, time_vars in course_time_vars.items():
            scheduled_times = [time_slot for time_slot, var in time_vars.items() if solver.Value(var)]
            for time_slot in scheduled_times:
                data.append({'Course ID': course_id, 'Scheduled Time': time_slot})
        schedule_df = pd.DataFrame(data)
        return schedule_df
    else:
        if status == cp_model.INFEASIBLE:
            print("No feasible solution found.")
        elif status == cp_model.MODEL_INVALID:
            print("The model is invalid. Please check the constraints and variables.")
        elif status == cp_model.UNKNOWN:
            print("The solver could not find a solution within the given time limit.")
        return pd.DataFrame(columns=['Course ID', 'Scheduled Time'])

In [ ]:
import pandas as pd
from collections import defaultdict

def check_conflicts(schedule_df, student_course_map):
    """
    Checks for scheduling conflicts for each student and returns a DataFrame
    containing all conflicts with details.

    Parameters:
    - schedule_df (pd.DataFrame): DataFrame containing the scheduled courses with their time slots.
    - student_course_map (dict): A mapping of student roll numbers to their enrolled courses.

    Returns:
    - pd.DataFrame: A DataFrame with columns ['Roll No.', 'Conflict Time Slot', 'Conflicting Courses']
                    listing all the conflicts for each student.
    """

    schedule = schedule_df.groupby('Course ID')['Scheduled Time'].apply(list).to_dict()

    conflict_rows = []
    for student, courses in student_course_map.items():
        time_slot_courses = defaultdict(list)
        unscheduled_courses = []
        for course in courses:
            scheduled_times = schedule.get(course)
            if scheduled_times:
                for time_slot in scheduled_times:
                    time_slot_courses[time_slot].append(course)
            else:
                unscheduled_courses.append(course)
        for time_slot, course_list in time_slot_courses.items():
            if len(course_list) > 1:
                # conflict
                conflict_rows.append({
                    'Roll No.': student,
                    'Conflict Time Slot': time_slot,
                    'Conflicting Courses': ', '.join(course_list)
                })
    conflict_df = pd.DataFrame(conflict_rows, columns=['Roll No.', 'Conflict Time Slot', 'Conflicting Courses'])
    return conflict_df


def find_courses_with_multiple_slots_on_same_day(schedule_df):
    """
    Identifies courses that are scheduled more than once on the same day and lists them.
    """
    def get_day_from_time_slot(time_slot):
        """
        Extracts the day from the time slot string.
        """
        return time_slot.split()[0]

    day_course_map = defaultdict(lambda: defaultdict(int))
    
    for _, row in schedule_df.iterrows():
        day = get_day_from_time_slot(row['Scheduled Time'])
        course_id = row['Course ID']
        day_course_map[day][course_id] += 1

    courses_with_multiple_slots = defaultdict(list)
    for day, course_counts in day_course_map.items():
        for course_id, count in course_counts.items():
            if count > 1:
                courses_with_multiple_slots[day].append(course_id)

    return courses_with_multiple_slots

In [ ]:
def gen_timetable(df_registration, df_courses, df_faculty_pref):
    
    df_merged = merge_data(df_registration, df_courses)
    student_course_map = prepare_student_course_map(df_merged)
    course_professor_map = create_course_professor_map(df_merged)
    professor_busy_slots = faculty_busy_slots(df_faculty_pref)
    course_credit_map = create_course_credit_map(df_courses)
    # Prepare courses and scheduleS
    courses = create_course_dictionary(student_course_map, course_professor_map, professor_busy_slots)
    schedule_data = schedule_courses(courses, student_course_map, course_professor_map, course_credit_map)
    print("Schedule Data")
    print()
    print(schedule_data)
    print()
    print("Courses on the same day")
    print(find_courses_with_multiple_slots_on_same_day(schedule_data))
    return schedule_data, check_conflicts(schedule_data, student_course_map)

In [ ]:
courses = pd.read_csv(r"/kaggle/input/timetable-courses/Course Data - Course Data (1).csv")
registration = pd.read_csv(r"/kaggle/input/timetable-courses/Course Data - Registration_TOP5.csv")
faculty_pref = pd.read_csv(r"/kaggle/input/timetable-courses/Course Data - Fac. Pref. (1).csv")



In [ ]:
import random
student_list = registration["Roll No."].tolist()
course_list = courses["Course code"].tolist()

schedule_data = gen_timetable(registration, courses, faculty_pref)
schedule_data.to_csv("Timetable.csv", index = None) 